# Face Recognition Attendance System


## Import the prerequisite libraries

In [5]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from tensorflow.keras.layers import Layer, InputSpec
from keras import backend as K
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from utils import LRN2D
import utils
import time, datetime
import csv
import urllib,json
from tkinter import * 
import parser
import tkinter.messagebox
import webbrowser
from tkinter import simpledialog
import tkinter.ttk as ttk
import csv

In [6]:
attendance_names = []
attendance_time = []
marked_students = {'none','null'}

d = datetime.datetime.today()
name_of_csv = "attendance/"+"{}-{}-{}".format(d.day,d.month,d.year) + '.csv'

## Here data about students is fetched from firebase-database

In [7]:
students = []
students_on_leave = []
student_names_dict = {}
d = datetime.datetime.today()
current_date = datetime.date(d.year,d.month,d.day)

with urllib.request.urlopen("https://leavemanagementsystem-8c5d3-default-rtdb.firebaseio.com/StudentsData.json") as url:
    student_data = json.loads(url.read())
    students = list(student_data.keys())
    print(student_data)
    print(students)
    for key in student_data:
        student_names_dict[student_data[key]['uid']] = student_data[key]['name']
    print('\n'+'#'*70+'\n')   
    print(student_names_dict)

print('-'*70)

with urllib.request.urlopen("https://leavemanagementsystem-8c5d3-default-rtdb.firebaseio.com/Leaves.json") as url:
    student_data = json.loads(url.read())
    print(student_data)
    for key in student_data:
#         print(key+ str(type(key)))
        for key1 in student_data[key].keys():
#             for key2 in key1:
#             print(key1 + str(type(key1)))
            year,month,day = int(student_data[key][key1]["endDate"][0:4]),int(student_data[key][key1]["endDate"][5:7]),int(student_data[key][key1]["endDate"][8:10])
            endDate = datetime.date(year,month,day)
            if ((endDate >=current_date) and (student_data[key][key1]["status"].lower()=='Approved'.lower())):
                students_on_leave.append(key)
#             print(student_data[key][key1]["endDate"])
#             print(student_data[key][key1]["status"])

{'19BCS6021': {'dob': '2000-05-25', 'email': 'amanbabbar@gmail.com', 'name': 'Aman Babbar', 'password': 'aman', 'uid': '19BCS6021'}, '19BCS6054': {'dob': '2000-02-18', 'email': 'jainsamyak9@gmail.com', 'name': 'Samyak Jain', 'password': 'Sam', 'uid': '19BCS6054'}, '19BCS6071': {'dob': '2001-11-21', 'email': 'ritikachopra123@gmail.com', 'name': 'Ritika Chopra', 'password': 'rits', 'uid': '19BCS6071'}, '19BCS6161': {'dob': '1999-11-23', 'email': 'manavgarg650@gmail.com', 'name': 'Manav Garg', 'password': 'abcd1234', 'uid': '19BCS6161'}}
['19BCS6021', '19BCS6054', '19BCS6071', '19BCS6161']

######################################################################

{'19BCS6021': 'Aman Babbar', '19BCS6054': 'Samyak Jain', '19BCS6071': 'Ritika Chopra', '19BCS6161': 'Manav Garg'}
----------------------------------------------------------------------
{'19BCS6021': {'23-4-2021_19-21-20': {'LeaveType': 'General', 'endDate': '2021-04-30', 'reason': 'Test', 'startDate': '2021-04-24', 'status': 'Appro

In [8]:
# print(student_data)
print(students_on_leave)
print(students)

[]
['19BCS6021', '19BCS6054', '19BCS6071', '19BCS6161']


### This function will mark attendance of the student whose face is recognized

In [9]:
time_now = []
def attendance_mark(name):
    d = datetime.datetime.today()
    time_now = "{}:{}:{}".format(d.hour,d.minute,d.second)
    if name in students:
        if name not in attendance_names:
            attendance_names.append(name)
            attendance_time.insert(attendance_names.index(name),time_now)
        else:
            pass

In [10]:
# attendance_time[0] = '22:34:5'
# # attendance_time[3] = '22:01:6'
# print(attendance_time)

### This function will check the name of students whose attendance is already marked

In [11]:
def check_marked_students():
    with open(name_of_csv,'r',newline='') as file:
        reader = csv.reader(file,delimiter=',')
        for line in reader:
#             print(line)
#             print(line[0])
            marked_students.add(line[0])
#             print(marked_students)
    
#     write_remaining_attendance()

# check_marked_students()
print(marked_students)

{'null', 'none'}


In [12]:
# print(attendance_time[0][3:5].replace(':',''))

### This function will write the 'present' of students whose face is recognised

In [13]:
def write_csv():
    with open(name_of_csv,'w',newline='') as file:
        
        writer = csv.writer(file)
        writer.writerow(['UID','Name','Status','Time'])
        
        for i in range(len(attendance_names)):            
            if (int(attendance_time[i][3:5].replace(':',''))<30 ): 
                grade = 'Present' 
            else: 
                grade = 'Late'
            writer.writerow([attendance_names[i],student_names_dict[attendance_names[i]],grade,attendance_time[i]])

        
#         writer = csv.writer(file)
#         for student in students_on_leave:
#              writer.writerow([student,'Leave'])
                
# write_csv()

### This function will mark 'absent' or 'leave' of remaining students

In [14]:
def write_remaining_attendance():
    with open(name_of_csv,'a',newline='') as file:
        print(marked_students)
        writer = csv.writer(file)
        
        for student in students_on_leave:
            if student not in marked_students:
                writer.writerow([student,student_names_dict[student],'Leave'])
                
        
        for student in students:
            if student not in marked_students:
                if student not in students_on_leave:
                    writer.writerow([student,student_names_dict[student],'Absent'])
                
        

# write_remaining_attendance()

### This function will rename the face picture of new student with their UID

In [15]:
filename_list = ["images/User_" + str(i) + ".jpg" for i in range(1,11)]
# print(filename_list)

def delete_and_rename(name):
    for file in filename_list:
        if (file=='images/User_10.jpg'):
            if os.path.exists(file):
                newname = 'images/'+name+'.jpg'
                print(newname)
                os.rename(file,newname)
        else:
            if os.path.exists(file):
                os.remove(file)
                
# delete_and_rename('19BCS6161')

# Neural Network Model
This model is based on FaceNet's Inception model.

In [16]:
myInput = Input(shape=(96, 96, 3))

x = ZeroPadding2D(padding=(3, 3), input_shape=(96, 96, 3))(myInput)
x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn1')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)
x = Lambda(LRN2D, name='lrn_1')(x)
x = Conv2D(64, (1, 1), name='conv2')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn2')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = Conv2D(192, (3, 3), name='conv3')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn3')(x)
x = Activation('relu')(x)
x = Lambda(LRN2D, name='lrn_2')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)

# Inception3a
inception_3a_3x3 = Conv2D(96, (1, 1), name='inception_3a_3x3_conv1')(x)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn1')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)
inception_3a_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3a_3x3)
inception_3a_3x3 = Conv2D(128, (3, 3), name='inception_3a_3x3_conv2')(inception_3a_3x3)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn2')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)

inception_3a_5x5 = Conv2D(16, (1, 1), name='inception_3a_5x5_conv1')(x)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn1')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)
inception_3a_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3a_5x5)
inception_3a_5x5 = Conv2D(32, (5, 5), name='inception_3a_5x5_conv2')(inception_3a_5x5)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn2')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)

inception_3a_pool = MaxPooling2D(pool_size=3, strides=2)(x)
inception_3a_pool = Conv2D(32, (1, 1), name='inception_3a_pool_conv')(inception_3a_pool)
inception_3a_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_pool_bn')(inception_3a_pool)
inception_3a_pool = Activation('relu')(inception_3a_pool)
inception_3a_pool = ZeroPadding2D(padding=((3, 4), (3, 4)))(inception_3a_pool)

inception_3a_1x1 = Conv2D(64, (1, 1), name='inception_3a_1x1_conv')(x)
inception_3a_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_1x1_bn')(inception_3a_1x1)
inception_3a_1x1 = Activation('relu')(inception_3a_1x1)

inception_3a = concatenate([inception_3a_3x3, inception_3a_5x5, inception_3a_pool, inception_3a_1x1], axis=3)

# Inception3b
inception_3b_3x3 = Conv2D(96, (1, 1), name='inception_3b_3x3_conv1')(inception_3a)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn1')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)
inception_3b_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3b_3x3)
inception_3b_3x3 = Conv2D(128, (3, 3), name='inception_3b_3x3_conv2')(inception_3b_3x3)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn2')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)

inception_3b_5x5 = Conv2D(32, (1, 1), name='inception_3b_5x5_conv1')(inception_3a)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn1')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)
inception_3b_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3b_5x5)
inception_3b_5x5 = Conv2D(64, (5, 5), name='inception_3b_5x5_conv2')(inception_3b_5x5)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn2')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)

inception_3b_pool = Lambda(lambda x: x**2, name='power2_3b')(inception_3a)
inception_3b_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: x*9, name='mult9_3b')(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_3b')(inception_3b_pool)
inception_3b_pool = Conv2D(64, (1, 1), name='inception_3b_pool_conv')(inception_3b_pool)
inception_3b_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_pool_bn')(inception_3b_pool)
inception_3b_pool = Activation('relu')(inception_3b_pool)
inception_3b_pool = ZeroPadding2D(padding=(4, 4))(inception_3b_pool)

inception_3b_1x1 = Conv2D(64, (1, 1), name='inception_3b_1x1_conv')(inception_3a)
inception_3b_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_1x1_bn')(inception_3b_1x1)
inception_3b_1x1 = Activation('relu')(inception_3b_1x1)

inception_3b = concatenate([inception_3b_3x3, inception_3b_5x5, inception_3b_pool, inception_3b_1x1], axis=3)

# Inception3c
inception_3c_3x3 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_3x3',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))

inception_3c_5x5 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))

inception_3c_pool = MaxPooling2D(pool_size=3, strides=2)(inception_3b)
inception_3c_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_3c_pool)

inception_3c = concatenate([inception_3c_3x3, inception_3c_5x5, inception_3c_pool], axis=3)

#inception 4a
inception_4a_3x3 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=192,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_4a_5x5 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(1, 1),
                                   padding=(2, 2))

inception_4a_pool = Lambda(lambda x: x**2, name='power2_4a')(inception_3c)
inception_4a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: x*9, name='mult9_4a')(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_4a')(inception_4a_pool)
inception_4a_pool = utils.conv2d_bn(inception_4a_pool,
                                   layer='inception_4a_pool',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   padding=(2, 2))
inception_4a_1x1 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_4a = concatenate([inception_4a_3x3, inception_4a_5x5, inception_4a_pool, inception_4a_1x1], axis=3)

#inception4e
inception_4e_3x3 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_3x3',
                                   cv1_out=160,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))
inception_4e_5x5 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_5x5',
                                   cv1_out=64,
                                   cv1_filter=(1, 1),
                                   cv2_out=128,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))
inception_4e_pool = MaxPooling2D(pool_size=3, strides=2)(inception_4a)
inception_4e_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_4e_pool)

inception_4e = concatenate([inception_4e_3x3, inception_4e_5x5, inception_4e_pool], axis=3)

#inception5a
inception_5a_3x3 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))

inception_5a_pool = Lambda(lambda x: x**2, name='power2_5a')(inception_4e)
inception_5a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: x*9, name='mult9_5a')(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_5a')(inception_5a_pool)
inception_5a_pool = utils.conv2d_bn(inception_5a_pool,
                                   layer='inception_5a_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   padding=(1, 1))
inception_5a_1x1 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))

inception_5a = concatenate([inception_5a_3x3, inception_5a_pool, inception_5a_1x1], axis=3)

#inception_5b
inception_5b_3x3 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_5b_pool = MaxPooling2D(pool_size=3, strides=2)(inception_5a)
inception_5b_pool = utils.conv2d_bn(inception_5b_pool,
                                   layer='inception_5b_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1))
inception_5b_pool = ZeroPadding2D(padding=(1, 1))(inception_5b_pool)

inception_5b_1x1 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_5b = concatenate([inception_5b_3x3, inception_5b_pool, inception_5b_1x1], axis=3)

av_pool = AveragePooling2D(pool_size=(3, 3), strides=(1, 1))(inception_5b)
reshape_layer = Flatten()(av_pool)
dense_layer = Dense(128, name='dense_layer')(reshape_layer)
norm_layer = Lambda(lambda  x: K.l2_normalize(x, axis=1), name='norm_layer')(dense_layer)


# Final Model
model = Model(inputs=[myInput], outputs=norm_layer)

## About image_to_embedding function        
When the model is loaded with pre trained weights, then we can create the **128 dimensional embedding vectors** for all the face images stored in the "images" folder. 

**"image_to_embedding"** function pass an image to the Inception network to generate the embedding vector.

In [17]:
def image_to_embedding(image, model):
    #image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_AREA) 
    image = cv2.resize(image, (96, 96)) 
    img = image[...,::-1]
    img = np.around(np.transpose(img, (0,1,2))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding


## About recognize_face function
This function calculate similarity between the captured image and the images that are already been stored. It passes the image to neural network to generate its embedding vector. Which is then compared with all the embedding vectors of the images stored by calculating Euclidean distance. 

If the minimum distance between two embeddings is less than a threshold then we have a match.

In [18]:
def recognize_face(face_image, input_embeddings, model):

    embedding = image_to_embedding(face_image, model)
    minimum_distance = 200
    name = None
    
    # Loop over  names and encodings.
    for (input_name, input_embedding) in input_embeddings.items():
        
       
        euclidean_distance = np.linalg.norm(embedding-input_embedding)
        

        print('Euclidean distance from %s is %s' %(input_name, euclidean_distance))
        attendance_mark(input_name)

        
        if euclidean_distance < minimum_distance:
            minimum_distance = euclidean_distance
            name = input_name
    
    if minimum_distance < 0.68:
        return str(name)
    else:
        return None

## About create_input_image_embeddings function
This function generates 128 dimensional image ebeddings of all the images stored in the "images" directory by forwarding the images to neural network. It creates a dictionary with key as the name of the face and value as embedding


## About recognize_faces_in_cam function
This function capture image from the webcam, detect a face in it and crop the image to have a face only, which is then passed to recognize_face function. 

In [19]:
import glob


def create_input_image_embeddings():
    input_embeddings = {}

    for file in glob.glob("images/*"):
        person_name = os.path.splitext(os.path.basename(file))[0]
        image_file = cv2.imread(file, 1)
        input_embeddings[person_name] = image_to_embedding(image_file, model)

    return input_embeddings



def recognize_faces_in_cam(input_embeddings):
    
    cv2.namedWindow("Face Recognizer")
    vc = cv2.VideoCapture(0)
   
    font = cv2.FONT_HERSHEY_SIMPLEX
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    while vc.isOpened():
        _, frame = vc.read()
        img = frame
        height, width, channels = frame.shape

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        # Loop through all the faces detected 
        identities = []
        for (x, y, w, h) in faces:
            x1 = x
            y1 = y
            x2 = x+w
            y2 = y+h

            face_image = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]    
            identity = recognize_face(face_image, input_embeddings, model)


            if identity is not None:
                img = cv2.rectangle(frame,(x1, y1),(x2, y2),(255,255,255),2)
                cv2.putText(img, str(identity), (x1+5,y1-5), font, 1, (255,255,255), 2)
        
        key = cv2.waitKey(100)
        cv2.imshow("Face Recognizer", img)

        if key == 27: # exit on ESC
            break
    vc.release()
    cv2.destroyAllWindows()

## Capturing the face image
This code captures 10 face images of the person. They all are stored in **"images"** folder with the name User_1 to User_10. Rename it with the name of person and delete rest of them. This image will be used for recognizing the the identity of the person.

In [20]:
def recognize_new_face(uid):
    cam = cv2.VideoCapture(0)

    face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

    count = 0
    while(True):
        ret, img = cam.read()
        #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_detector.detectMultiScale(img, 1.3, 5)
        for (x,y,w,h) in faces:
            x1 = x
            y1 = y
            x2 = x+w
            y2 = y+h
            cv2.rectangle(img, (x1,y1), (x2,y2), (255,255,255), 2)     
            count += 1
            # Save the captured image into the datasets folder
            cv2.imwrite("images/User_" + str(count) + ".jpg", img[y1:y2,x1:x2])
            cv2.imshow('image', img)

        k = cv2.waitKey(200) & 0xff # Press 'ESC' for exiting video
        if k == 27:
            break
        elif count >= 10: # Take 30 face sample and stop video
             break
    
    delete_and_rename(uid)
    cam.release()
    cv2.destroyAllWindows()

## This function will start the Face Recognizer program

In [21]:
def start_face_recognizer():
    input_embeddings = create_input_image_embeddings()
    recognize_faces_in_cam(input_embeddings)

In [22]:
# newname = input("Enter UID of student: ")
# recognize_new_face(newname)

In [23]:
# start_face_recognizer()

## GUI for Viewing Attendance

In [24]:
def view_attendance():
    root1 = Tk()
    root1.title(name_of_csv.split('/')[1])
    width = 500
    height = 400
    screen_width = root1.winfo_screenwidth()
    screen_height = root1.winfo_screenheight()
    x = (screen_width/2) - (width/2)
    y = (screen_height/2) - (height/2)
    root1.geometry("%dx%d+%d+%d" % (width, height, x, y))
    root1.resizable(0, 0)
    TableMargin = Frame(root1, width=500)
    TableMargin.pack(side=TOP)
    tree = ttk.Treeview(TableMargin, columns=("UID", "Name", "Status","Time"), height=400)
    tree.heading('UID', text="UID", anchor=W)
    tree.heading('Name', text="Name", anchor=W)
    tree.heading('Status', text="Status", anchor=W)
    tree.heading('Time', text="Time", anchor=W)
    tree.column('#0', stretch=NO, minwidth=0, width=0)
    tree.column('#1', stretch=NO, minwidth=0, width=150)
    tree.column('#2', stretch=NO, minwidth=0, width=150)
    tree.column('#3', stretch=NO, minwidth=0, width=100)
    tree.pack()
    with open(name_of_csv) as f:
        reader = csv.DictReader(f, delimiter=',')
        for row in reader:
            uid = row['UID']
            name = row['Name']
            status = row['Status']
            time = row['Time']
            tree.insert("", 0, values=(uid,name,status,time))
        
    root1.mainloop()

## This is main GUI visible to the user

In [25]:
from tkinter import * 
from tkinter import simpledialog
import parser
import tkinter.messagebox
import webbrowser

root = Tk()
root.title("FACE RECOGNITION ATTENDANCE SYSTEM")
root.configure(background="#800000")
root.resizable(width=False,height=False)
root.state('zoomed') #full screen

att=Frame(root,bg="#800000")
att.pack()


def recognize_new_face_button():
    USER_INP = simpledialog.askstring(title="New Student Registration", prompt="Enter UID")
    recognize_new_face(USER_INP)
    
sr_img = PhotoImage(file = "GUI Images/SR_BUTTON.png")
r=Button(att,image = sr_img,width=550,height=65,font=('times new roman',17,'bold'),
             bg="white",command=recognize_new_face_button).grid(row=2,column=1,padx=20, pady=80)


start_img = PhotoImage(file = "GUI Images/START_BUTTON.png")
start=Button(att,image = start_img,width=550,height=65,font=('times new roman',17,'bold'),
             bg="white",command=start_face_recognizer).grid(row=2,column=2,padx=20, pady=80)


def mark_attendance_button():
    msg=tkinter.messagebox.showinfo('Success!','Attendance has been marked.')
    write_csv()
    check_marked_students()
    write_remaining_attendance()

ma_img = PhotoImage(file = "GUI Images/MA_BUTTON.png")
ma=Button(att,image = ma_img,width=550,height=65,font=('times new roman',17,'bold'),border="0",
             bg="white",command=mark_attendance_button).grid(row=3,column=1,padx=20, pady=80)

va_img = PhotoImage(file = "GUI Images/VA_BUTTON.png")
va=Button(att,image = va_img,width=550,height=65,font=('times new roman',17,'bold'),
             bg="white",command=view_attendance).grid(row=3,column=2,padx=20, pady=80)


def help_desk():
    msg=tkinter.messagebox.showinfo('Helpdesk','For any disperancies, contact "19BCS6161@cuchd.in"')

hd_img = PhotoImage(file = "GUI Images/HD_BUTTON.png")
hd=Button(att,image = hd_img,width=550,height=65,font=('times new roman',17,'bold'),
             bg="white",command=help_desk).place(relx=0.5, rely=0.5, anchor=CENTER)


#MENU BAR (EXIT AND HELP)

menubar=Menu(att)

def exit():
    exit=tkinter.messagebox.askyesno("ATTENDANCE MANAGEMENT SYSTEM","DO YOU WANT TO EXIT?")
    if exit>0 :
        root.destroy()
        return

filemenu=Menu(menubar,tearoff=0)
menubar.add_cascade(label="EXIT",menu=filemenu)
filemenu.add_command(label="Exit",command=exit)


helpmenu=Menu(menubar,tearoff=0)
menubar.add_cascade(label="HELP",menu=helpmenu)
helpmenu.add_command(label='Contact "19BCS6161@cuchd.in"')

root.config(menu=menubar)

root.mainloop()